# BERT tutorial using Hugging Face
## 教學目標
利用 Hugging Face 套件快速使用 BERT 模型來進行下游任務訓練
- 單一句型分類任務 (single sentence text classification)
- 成對句型分類任務 (sentence pair text classification)
- 問答任務 (question answering)
- 命名實體辨識 (named-entity recognition / token-level text classification)

# 單一句型分類任務
## 下載資料
我們使用 IMDb reviews 資料集作為範例

In [1]:
!pip install wget
import wget
url = 'http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz'
filename = wget.download(url, out='./')


In [3]:
import tarfile

# 指定檔案位置，並解壓縮 .gz 結尾的壓縮檔
tar = tarfile.open('aclImdb_v1.tar.gz', 'r:gz')
tar.extractall()

## 接下來我們要進行資料前處理
但首先要觀察解壓縮後的資料夾結構:
```
aclImdb---
        |--train
        |    |--neg
        |    |--pos
        |    |--...
        |--test
        |    |--neg
        |    |--pos
        |    |--...
        |--imdb.vocab
        |--imdbEr.text
        |--README
```
其中train和test資料夾中分別又有neg和pos兩種資料夾

我們要針對這兩個目標資料夾進行處理

In [21]:
# 載入 pathlib 模組 (Python3.4+)
from pathlib import Path

def read_imdb_split(split_dir):
    split_dir = Path(split_dir)
    texts = []
    labels = []
    for label_dir in ["pos", "neg"]:
        # 利用iterdir() 來列出資料夾底下的所有檔案，此功能等同於 os.path.listdir()
#         for text_file in (split_dir/label_dir).iterdir():
        # 若知道副檔名為.txt
        for text_file in (split_dir/label_dir).glob("*.txt"):
            tmp_text = text_file.read_text()
#             print(tmp_text)
#             exit()
            texts.append(tmp_text)
            labels.append(0 if label_dir == "neg" else 1)
    
    return texts, labels

In [22]:
train_texts, train_labels = read_imdb_split('aclImdb/train')
test_texts, test_labels = read_imdb_split('aclImdb/test')

### 切分訓練資料，來分出 validation set

In [25]:
from sklearn.model_selection import train_test_split

# 設立隨機種子來控制隨機過程
random_seed = 42

# 設定要分出多少比例的 validation data
valid_ratio = 0.2

# 使用 train_test_split 來切分資料
train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=valid_ratio, random_state=random_seed)

### Tokenizization
- 斷字的部份以 DistilBERT (Sanh et al., 2019) 的 tokenizer 為例
- Hugging Face 的 tokenizer 可以直接幫你自動將資料轉換成 BERT 的輸入型式 (也就是加入[CLS]和[SEP] tokens)
- 接著由於深度學習網路需要使用 mini-batch 進行學習，我們需要先以 PyTorch dataset 來自行建立封裝資料的物件

In [26]:
from transformers import DistilBertTokenizerFast

# 在 Hugging Face 套件中可使用 .from_pretrained() 的方法來導入預訓練模型
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

In [27]:
# 分別將3種資料 (train/valid/test) 做 tokenization
# truncation 代表

train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)
test_encodings = tokenizer(test_texts, truncation=True, padding=True)

In [29]:
import torch

class IMDbDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = IMDbDataset(train_encodings, train_labels)
val_dataset = IMDbDataset(val_encodings, val_labels)
test_dataset = IMDbDataset(test_encodings, test_labels)